O projeto deve conter:
- arquivo README.md, requirements.txt, diretórios notebook e arquivo app.py - FEITO
- link para acesso remoto (Render) - FEITO
- pelo menos um cabeçalho com texto - FEITO
- pelo menos 1 histograma - FEITO 
- pelo menos 1 gráfico de dispersão
- pelo menos um botão ou uma caixa de seleção - FEITO

DESAFIO: SUBSTITUIR BOTÕES POR CAIXA DE SELEÇÃO ONDE O USUÁRIO SELECIONA HIST OU DISPERÇÃO - FEITO

In [ ]:
# importando bibliotecas
import pandas as pd
import plotly.express as px
import streamlit as st
from pandas.api.types import is_numeric_dtype

#### Criação, edição e leitura do df

In [ ]:
# criando df e lendo os dados
df_vehicles = pd.read_csv('../vehicles_us.csv') 
car_data = df_vehicles.copy()
# car_data.info()
car_data.sample(10)
# car_data.describe(include='all')
# dropando algumas colunas que não me interessam
car_data = car_data.drop(columns=["days_listed", "date_posted"])
# criando duas novas colunas, uma apenas com a marca e outra apenas com o nome do carro
car_data[["brand", "model_name"]] = car_data["model"].str.split(
    pat=" ", n=1, expand=True
)
car_data.info()

In [ ]:
# car_data["transmission"].value_counts()
# car_data["cylinders"].value_counts()
# car_data["odometer"].describe().round(2)
car_data.sample(10)

#### Select box com opções de colunas do df para criação dos histogramas

In [ ]:
st.subheader("Histogramas")

feature_map = {
    "Ano de Fabricação": "model_year",
    "Tipo de Carro": "type",
    "Cor do Carro": "paint_color",
    "Marcas": "brand",
}

feature_label = st.selectbox(
    "Escolha uma variável para visualizar:",
    list(feature_map.keys()),
)

column_name = feature_map[feature_label]
series = car_data[column_name].dropna()

if is_numeric_dtype(series):
    fig = px.histogram(
        car_data,
        x=column_name,
        nbins=30,
        title=f"Histograma de {feature_label}",
    )
    fig.update_layout(xaxis_title=None)
else:
    fig = px.histogram(
        car_data,
        x=column_name,
        color=column_name,
        title=f"Distribuição de {feature_label}",
    )
    fig.update_layout(xaxis_title=None)

st.plotly_chart(fig, use_container_width=True)
st.caption("Os valores nulos foram removidos para geração dos gráficos acima.")
st.divider()

#### Botão que gera gráfico de dispersão

In [ ]:
def gerar_dispersao(df, x_col, y_col):
    """Função que cria um scatterplot baseado no clique do usuário em um botão e o adiciona à tela.
    Opções de gráfico:
    - quilometragem vs preço;
    - ano de fabricação vs preço.
    """
    fig = px.scatter(
        df,
        x=x_col,
        y=y_col,
        opacity=0.7,
        trendline="ols",
        # color=color_var,       # comente/retire se não quiser cores
        # size=size_var,         # comente/retire se não quiser tamanho variável
        title=f"{x_col} vs {y_col}"
    )
    
    # atualizando o eixo x da quilometragem de milhões para milhares 
    if x_col == "odometer":
        fig.update_xaxes(
            tickformat=",d",
            separatethousands=True
    )
    
    # exibindo no Streamlit:
    st.plotly_chart(fig, use_container_width=True)

st.subheader("Gráficos de dispersão")

# algorítmo abaixo não funcionou no streamlit, alterei para if para que o gráfico fosse renderizado na tela
# botao_odo_vs_price = st.button("Quilometragem vs Preço", on_click=lambda: gerar_dispersao(car_data, "odometer", "price"))
# botao_year_vs_price = st.button("Ano de Fabricação vs Preço", on_click=lambda: gerar_dispersao(car_data, "model_year", "price"))

# código atualizado -> o layout ficava estranho, com o gráfico aparecendo entre os botões, atualizei novamente (próximo código):
# if st.button("Quilometragem vs Preço"):
#     gerar_dispersao(car_data, "odometer", "price")    
# if st.button("Ano de Fabricação vs Preço"):
#     gerar_dispersao(car_data, "model_year", "price")    

# ultima atualização -> usando SESSION STATES:
# inicializando o session state
if "grafico_atual" not in st.session_state:
    st.session_state["grafico_atual"] = None

# adicionando e mantendo os botões lado a lado
button_1, button_2 = st.columns(2)

with button_1:
    if st.button("Quilometragem vs Preço"):
        st.session_state["grafico_atual"] = ("odometer", "price")
with button_2:
    if st.button("Ano de Fabricação vs Preço"):
        st.session_state["grafico_atual"] = ("model_year", "price")

# atualizando o session state para mostrar o gráfico na tela
if st.session_state["grafico_atual"] is not None:
    x_col, y_col = st.session_state["grafico_atual"]
    gerar_dispersao(car_data, x_col, y_col)
    
    
st.divider()